In [13]:
from sklearn.datasets import load_files
import numpy as np


reviews_train = load_files("data/aclImdb/train/")
reviews_test = load_files("data/aclImdb/test/")

In [ ]:
# 7.2. Example Application: Sentiment Analysis of Movie Reviews


text_train, y_train = reviews_train.data, reviews_train.target
print(f"text_train type: {type(text_train)}")
print(f"text_train len: {len(text_train)}")
print(f"text_train[1]:\n{text_train[1]}")

text_train type: <class 'list'>
text_train len: 75000
text_train[1]:
b"Amount of disappointment I am getting these days seeing movies like Partner, Jhoom Barabar and now, Heyy Babyy is gonna end my habit of seeing first day shows.<br /><br />The movie is an utter disappointment because it had the potential to become a laugh riot only if the d\xc3\xa9butant director, Sajid Khan hadn't tried too many things. Only saving grace in the movie were the last thirty minutes, which were seriously funny elsewhere the movie fails miserably. First half was desperately been tried to look funny but wasn't. Next 45 minutes were emotional and looked totally artificial and illogical.<br /><br />OK, when you are out for a movie like this you don't expect much logic but all the flaws tend to appear when you don't enjoy the movie and thats the case with Heyy Babyy. Acting is good but thats not enough to keep one interested.<br /><br />For the positives, you can take hot actresses, last 30 minutes, some com

In [14]:
text_train = [doc.replace(b"<br />", b" ") for doc in text_train]
print(f"Samples per class(train): {np.bincount(y_train)}")

Samples per class(train): [12500 12500 50000]


In [15]:
text_test, y_test = reviews_test.data, reviews_test.target
print(f"Number of documents in the test data: {len(text_test)}")
print(f"Samples per class(test): {np.bincount(y_test)}")
text_test = [doc.replace(b"<br />", b" ") for doc in text_test]

Number of documents in the test data: 25000
Samples per class(test): [12500 12500]
